In [1]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

## List up attractions using LLM

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import Optional
llm = ChatOpenAI(model='gpt-3.5-turbo-16k',temperature=0.1)

In [4]:
template = """\
## Instruction
List up {k} attractions in {place}.
{user_profile}

## Output
{format_instruction}
"""

In [5]:
class Attraction(BaseModel):
    name: str = Field(description='The name of attraction')
    country: str = Field(description='The country the attraction located')
    city: str = Field(description='The city the attraction located')

    description: str = Field(description='Description of the attraction. More than 1000 words.')
    duration: float = Field(description='Duration time of the attraction. Hours.')

    lat: Optional[float] = Field(description='latitude of the attraction')
    lng: Optional[float] = Field(description='longitude of the attraction')

class Attractions(BaseModel):
    data: list[Attraction] = Field(description='List of Atrractions')

parser = PydanticOutputParser(pydantic_object=Attractions)

In [6]:
prompt = ChatPromptTemplate.from_template(
    template=template,
    partial_variables={'format_instruction':parser.get_format_instructions()},
    output_parser=parser
)

In [7]:
chain = LLMChain(llm=llm,prompt=prompt,output_parser=parser)#,verbose=True)

In [8]:
user_profile = "This is my first visit in here."
res = chain.run(place='London',k=3,user_profile=user_profile)

In [9]:
data = res.dict()['data']

In [10]:
data[0]

{'name': 'The British Museum',
 'country': 'United Kingdom',
 'city': 'London',
 'description': 'The British Museum is a museum dedicated to human history, art, and culture. It houses a vast collection of works from all continents, spanning over two million years of history.',
 'duration': 3.0,
 'lat': 51.5194,
 'lng': -0.1269}

## Get latitude and longitude using GoogleMaps API (Not needed)

In [29]:
import googlemaps,os,json
gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

In [20]:
for i,d in enumerate(data):
    print(i,d['name'],'-'*30)
    geocode_result = gmaps.geocode(d['name'])
    gmap_res = geocode_result[0]["geometry"]["location"]
    print('GoogleMaps:',gmap_res["lat"],gmap_res["lng"])
    print('ChatGPT:',d["lat"],d["lng"])
    d['lat'],d['lng'] = gmap_res['lat'],gmap_res['lng']

0 The British Museum ------------------------------
GoogleMaps: 51.5194133 -0.1269566
ChatGPT: 51.5194 -0.1269
1 Tower of London ------------------------------
GoogleMaps: 51.50811239999999 -0.0759493
ChatGPT: 51.5081 -0.0761
2 Buckingham Palace ------------------------------
GoogleMaps: 51.501364 -0.14189
ChatGPT: 51.5014 -0.1419


## Get directions using GoogleMaps API

In [34]:
# https://qiita.com/shin1007/items/8f31188b9ac83be9e738
def decode_polyline(enc: str):
    """
    Parameters
    ----------
    enc : str
        encoded string of polyline, which can be aquired via Google Maps API.

    Returns
    -------
    result : list
        each element in `result` contains pair of latitude and longitude.
    """
    if enc == None or enc == '':
        return [[0, 0]]

    result = []
    polyline_chars = list(enc.encode())
    current_latitude = 0
    current_longitude = 0
    try:
        index = 0
        while index < len(polyline_chars):
            # calculate next latitude
            total = 0
            shifter = 0

            while True:
                next5bits = int(polyline_chars[index]) - 63
                index += 1
                total |= (next5bits & 31) << shifter
                shifter += 5
                if not(next5bits >= 32 and index < len(polyline_chars)):
                    break

            if (index >= len(polyline_chars)):
                break

            if((total & 1) == 1):
                current_latitude += ~(total >> 1)
            else:
                current_latitude += (total >> 1)

            # calculate next longitude
            total = 0
            shifter = 0
            while True:
                next5bits = int(polyline_chars[index]) - 63
                index += 1
                total |= (next5bits & 31) << shifter
                shifter += 5
                if not(next5bits >= 32 and index < len(polyline_chars)):
                    break

            if (index >= len(polyline_chars) and next >= 32):
                break

            if((total & 1) == 1):
                current_longitude += ~(total >> 1)
            else:
                current_longitude += (total >> 1)

            # add to return value
            pair = [current_latitude / 100000, current_longitude / 100000]
            result.append(pair)

    except:
        pass
    return result

In [76]:
from datetime import datetime, timedelta
now = datetime.strptime('2023/08/01 09:00','%Y/%m/%d %H:%M')

routes = []
first_trans_min = 30
print(now.strftime('%H:%M'), f"Going to {data[0]['name']}")
print('  ',f'Duration {str(int(first_trans_min))} minutes')
now += timedelta(minutes=first_trans_min)

for i in range(len(data)-1):
    # print(now)
    directions_result = gmaps.directions(
        data[i]['name'],
        data[i+1]['name'],
        mode="transit",
        departure_time=now
    )
    # print('directions_result:',directions_result[0]['legs'][0]['duration'])#.keys())
    duration_attraction_hour = data[i]['duration']
    print(now.strftime('%H:%M'), data[i]['name'])
    print('  ',f'Spent {"{:.1f}".format(duration_attraction_hour)} hours')
    now += timedelta(hours=duration_attraction_hour)
    
    duration_direction_sec = directions_result[0]['legs'][0]['duration']['value']
    print(now.strftime('%H:%M'), f"Going to {data[i]['name']}")
    print('  ',f'Duration {str(int(duration_direction_sec/60))} minutes')
    now += timedelta(seconds=duration_direction_sec)
    
    # print(now)
    steps = []
    for j,s in enumerate(directions_result[0]['legs'][0]['steps']):
        # print(j,'-'*30)
        steps.append({
            'polyline': decode_polyline(s['polyline']['points']),
            'travel_mode': s['travel_mode'],
            'duration': s['duration']['text'],
            'duration_sec': s['duration']['value']
        })
    routes.append(steps)
    # break
print(now.strftime('%H:%M'), "End")

09:00 Going to The British Museum
   Duration 30 minutes
09:30 The British Museum
   Spent 3.0 hours
12:30 Going to The British Museum
   Duration 27 minutes
12:57 Tower of London
   Spent 2.0 hours
14:57 Going to Tower of London
   Duration 38 minutes
15:35 End


In [57]:
import folium
from branca.element import Figure
import pandas as pd
import numpy as np

color_map = {
    'WALKING': '#0000ff',
    'TRANSIT': '#ff0000',
}

In [77]:
lat = np.mean([d['lat'] for d in data],)
lng = np.mean([d['lng'] for d in data],)
map = folium.Map(location=[lat,lng], zoom_start=14)

for d in data:
    folium.Marker(location=[d['lat'], d['lng']], popup=f"{d['name']} ({d['duration']})").add_to(map)

for r in routes:
    for step in steps:
        folium.vector_layers.PolyLine(
            locations=step['polyline'], popup=f'{step["travel_mode"]} ({step["duration"]})', color=color_map[step['travel_mode']]
        ).add_to(map)

fig = Figure(width=800, height=600)
fig.add_child(map)